In [30]:
# Installation de la librairie qui contient le modèle deeplabv3+ (utilisable seulement sur collab)
!pip install segmentation-models-pytorch==0.2.0,

In [32]:
from segmentation_models_pytorch import DeepLabV3Plus
print("Deeplabv3Plus importé avec succes !")

Deeplabv3Plus importé avec succes !


**Modifiez par le dossier contenant tout le code**

In [31]:
path = "/content/drive/MyDrive/Deeplabv3+_RoadSegmentation/"  # Chemin du modèle


In [33]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from torchvision import transforms

def overlay_mask(original_image, binary_mask, color=[255, 0, 0], alpha=0.5):
    """
    Superpose un masque binaire coloré sur l'image originale sans assombrir les zones non concernées.
    """
    binary_mask_resized = Image.fromarray(binary_mask * 255).resize(original_image.size)
    original_array = np.array(original_image)
    mask_array = np.array(binary_mask_resized) / 255.0

    overlay = np.zeros_like(original_array, dtype=np.uint8)
    overlay[:] = color  # Remplir toute l'image avec la couleur

    blended = original_array.copy()
    blended[mask_array > 0] = (
        (1 - alpha) * blended[mask_array > 0] + alpha * overlay[mask_array > 0]
    ).astype(np.uint8)

    return Image.fromarray(blended)
def preprocess_image(image_path):
    """
    Charge et prétraite une image.
    """
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0), image

def load_model(weights_path):
    """
    Charge un modèle pré-entraîné.
    """
    model = torch.load(weights_path, map_location=torch.device("cpu"))
    model.eval()
    print("Modèle chargé avec succès.")
    return model

def perform_segmentation(folder_path, model, threshold=0.7, save_dir="segmented_images", color=[255, 0, 0]):
    """
    Effectue la segmentation sur toutes les images d'un dossier.
    """
    os.makedirs(save_dir, exist_ok=True)
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image_tensor, original_image = preprocess_image(image_path)

            with torch.no_grad():
                output = model(image_tensor)
                output_mask = torch.sigmoid(output[0][0]).numpy()

            binary_mask = (output_mask > threshold).astype(np.uint8)
            inverted_mask = 1 - binary_mask  # Inversion du masque
            blended_image = overlay_mask(original_image, binary_mask, color=color)

            mask_image = Image.fromarray((inverted_mask * 255).astype(np.uint8))
            mask_path = os.path.join(save_dir, f"{os.path.splitext(filename)[0]}_mask.png")
            mask_image.save(mask_path)
            print(f"Masque inversé sauvegardé : {mask_path}")

            # Création de la carte de probabilité
            plt.figure(figsize=(6, 6))
            plt.imshow(output_mask, cmap='jet', alpha=0.6)
            plt.colorbar()
            plt.axis("off")
            plt.title("Carte de probabilité de segmentation")
            plt.savefig("temp_prob_map.png")
            plt.show()
            plt.close()

            # Chargement de la carte de probabilité et concaténation avec l'image segmentée
            prob_map = Image.open("temp_prob_map.png").resize(blended_image.size)
            concatenated = Image.new('RGB', (blended_image.width * 2, blended_image.height))
            concatenated.paste(blended_image, (0, 0))
            concatenated.paste(prob_map, (blended_image.width, 0))

            save_path = os.path.join(save_dir, f"{os.path.splitext(filename)[0]}_concatenated.png")
            concatenated.save(save_path)
            print(f"Image concaténée sauvegardée : {save_path}")

# Exécution
weights_path = path+"best_model.pth"
folder_path = path+"BDD-7227"
save_dir = path+"images_test"
model = load_model(weights_path)
perform_segmentation(folder_path, model, threshold=0.7, save_dir=save_dir, color=[0, 255, 0])


Output hidden; open in https://colab.research.google.com to view.